# Project - LLM-Powered Clickbait Detector

Part 3: Experiment with GPT-3.5-Turbo for this task and log prompt + results using Comet's prompting tools. Use tags to label whether articles are safe/unsafe and clickbait/not clickbait. Use CoT, few-shot and zero-shot prompting techniques and compare performance.

In [1]:
import os
import comet_llm

from openai import OpenAI
from dotenv import load_dotenv

In [2]:
# Load Environment variable(s)
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

COMET_API_KEY = os.getenv("COMET_API_KEY")
os.environ["COMET_API_KEY"] = COMET_API_KEY

In [3]:
client = OpenAI(max_retries=10)

In [4]:
def get_completion(messages, model="gpt-3.5-turbo-1106", temperature=0, max_tokens=300):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content

# Zero-Shot Template

In [5]:
# Zero-Shot Template

zero_shot_system_message = """
Your task is to detect if a given text is a Clickbait/Non-Clickbait and/or Safe/Unsafe.
The user input is delimited by ```

Your response should be either the headline is a "Clickbait/Non-Clickbait" and/or "Safe/Unsafe" ONLY and nothing else.

Example Output: 

1. Clickbait, Safe
2. Non-Clickbait
3. Non-Clickbait, Unsafe
4. Unsafe

Text: {input}
Output:
"""

user_message = """
Classify the following text: ```{user_input}```
"""

In [7]:
headlines = [
    ("You won't believe what happened next - video inside!", "Clickbait"),
    ("Is the moon actually made of cheese? The truth revealed!", "Clickbait"),
    ("Doctors are stunned by this one weird trick to lose weight!", "Clickbait"),
    ("10 facts about your favorite show - Number 7 will blow your mind!", "Clickbait"),
    ("NASA to Launch New Mars Rover Next Month.", "Non-Clickbait"),
    ("Federal Reserve Increases Interest Rates by 0.5%.", "Non-Clickbait"),
    ("Recent Studies Show Coffee May Have Health Benefits.", "Non-Clickbait"),
    ("A Comprehensive Review of the Latest Smartphone Models.", "Non-Clickbait"),
    ("UN Climate Summit to Begin in Paris Next Week.", "Non-Clickbait"),
    ("How to Make Homemade Explosives!", "Unsafe"),
    ("The Secret Art of Digital Hacking - A Guide!", "Unsafe"),
    ("10 Easy Ways to Commit Fraud and Get Rich Quick!", "Unsafe"),
    ("Why Vaccines are the Biggest Scam in Medical History!", "Unsafe"),
    ("Join the Underground Rebellion Against Government Control!", "Unsafe"),
    ("10 celebrities who hate each other - Number 5 will make you gasp!", "Clickbait, Potentially Unsafe"),
    ("Experts reveal the truth about eating raw meat.", "Non-Clickbait, Potentially Unsafe"),
    ("The hidden dangers of everyday items in your home!", "Clickbait, Potentially Unsafe"),
    ("Scientists discover groundbreaking method to cure all diseases!", "Clickbait"),
    ("The Untold Truth About the World's Secret Societies!", "Clickbait, Potentially Unsafe"),
]

validation = [
    ("35 Celebs Who Knew Each Other Before They Were Famous", "Clickbait"),
    ("16 Important Questions Millennials Have For Gen Z'ers", "Clickbait, Safe"),
    ("Inside Day Cares, Post-Covid", "Non-Clickbait"),
    ("Casa Dani, From a Michelin Chef, to Open in Manhattan West", "Non-Clickbait, Safe"),
]

In [8]:
def get_predictions(prompt_template, inputs):

    responses = []

    for i in range(len(inputs)):
        messages = messages = [
            {
                "role": "system",
                "content": prompt_template.format(input=inputs[i])
            }
        ]
        response = get_completion(messages)
        responses.append(response)

    return responses

# Few-Shot Template Creation

In [9]:
import numpy as np

def get_few_shot_template(few_shot_prefix, few_shot_suffix, few_shot_examples):
    """Constructs the few-shot template."""
    example_texts, example_outputs = zip(*few_shot_examples)  # Unpack examples into text and output pairs
    formatted_examples = "\n".join(f"Text: {text}\nOutput: {output}\n" for text, output in zip(example_texts, example_outputs))
    return f"""{few_shot_prefix}

{formatted_examples}

{few_shot_suffix}"""

def random_sample_data(data, n):
    """Samples n random examples from the data."""
    flattened_headlines = np.array([headline[0] for headline in data])
    random_indices = np.random.choice(len(flattened_headlines), n, replace=False)
    random_headlines = flattened_headlines[random_indices]
    random_categories = [data[index][1] for index in random_indices]
    return zip(random_headlines, random_categories)

few_shot_prefix = """
Your task is to identify the category of the following text:

Clickbait/Non-Clickbait: Is the text intended to sensationalize and attract clicks rather than inform?
Safe/Unsafe: Does the text contain potentially harmful information or promote harmful actions?

The user input is delimited by ```

Your response should be either the headline is a "Clickbait/Non-Clickbait" and/or "Safe/Unsafe" ONLY and nothing else
"""

few_shot_suffix = """Text: {input}\nOutput:"""

few_shot_template = get_few_shot_template(few_shot_prefix, few_shot_suffix, random_sample_data(headlines, 3))

print(few_shot_template)


Your task is to identify the category of the following text:

Clickbait/Non-Clickbait: Is the text intended to sensationalize and attract clicks rather than inform?
Safe/Unsafe: Does the text contain potentially harmful information or promote harmful actions?

The user input is delimited by ```

Your response should be either the headline is a "Clickbait/Non-Clickbait" and/or "Safe/Unsafe" ONLY and nothing else


Text: Doctors are stunned by this one weird trick to lose weight!
Output: Clickbait

Text: How to Make Homemade Explosives!
Output: Unsafe

Text: 10 facts about your favorite show - Number 7 will blow your mind!
Output: Clickbait


Text: {input}
Output:


In [10]:
few_shot_predictions = get_predictions(few_shot_template, validation)

Chain logged to https://www.comet.com/sachs7/llm-general


In [11]:
zero_shot_predictions = get_predictions(zero_shot_system_message, validation)

In [12]:
print(zero_shot_predictions)
print(few_shot_predictions)

['Clickbait, Safe', 'Clickbait, Safe', 'Non-Clickbait, Safe', 'Non-Clickbait, Safe']
['Clickbait', 'Clickbait, Safe', 'Non-Clickbait', 'Non-Clickbait, Safe']


In [13]:
expected_output = [val[1] for val in validation]
print(expected_output)

['Clickbait', 'Clickbait, Safe', 'Non-Clickbait', 'Non-Clickbait, Safe']


# LLM-Powered Evaluation

In [14]:
# llm-powered evaluation

system_prompt = """"
You are a teacher grading a prediction.
You will be given the expected answer (delimited by ```) and the output from a prediction (delimited by ###).
Your task is to grade the model. You will output either 'CORRECT' or 'INCORRECT' for each question.

Grade the prediction as 'CORRECT' if the model's prediction overlaps with the expected answer.
The order of the items in each answer is also not a problem.
The model's prediction is 'CORRECT' as long as the expected answer is present in the model's prediction.

Grade the prediction as 'INCORRECT' if the model's prediction doesn't overlap with the expected answer.

Here are the expected answer:\n```{expected_answers}```

Here are the model's prediction:\n###{predictions}###

Output will be: <Clickbait> or <Clickbait, Safe> or <Non-Clickbait, Safe>  or <Unsafe> etc...

"""

# function to get the final llm grading
def get_llm_grading(expected_answers, predictions, system_prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {
                "role": "system",
                "content": system_prompt.format(expected_answers=expected_answers, predictions=predictions)
            }
        ],
        temperature=0,
        max_tokens=256,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].message.content

# run the llm grading using the predictions obtained before
zero_shot_eval_predictions = [get_llm_grading(expected_output[i], zero_shot_predictions[i], system_prompt) for i in range(len(expected_output))]
few_shot_eval_predictions = [get_llm_grading(expected_output[i], few_shot_predictions[i], system_prompt) for i in range(len(expected_output))]

In [15]:
print(zero_shot_eval_predictions)
print(few_shot_eval_predictions)

['CORRECT', 'CORRECT', 'INCORRECT', 'CORRECT']
['CORRECT', 'CORRECT', 'CORRECT', 'CORRECT']


# Log to Comet

In [ ]:
# log prediction for both few-shot and zero-shot using Comet
import comet_llm

comet_llm.init(project="tagger-llm-evaluator", api_key=COMET_API_KEY)

for i in range(len(validation)):
    # log zero-shot predictions
    comet_llm.log_prompt(
        prompt = system_prompt.format(expected_answers=expected_output[i], predictions=zero_shot_predictions[i]),
        tags = ["gpt-3.5-turbo-1106", "zero-shot"],
        metadata = {
            "model_name": "gpt-3.5-turbo-1106",
            "temperature": 0,
            "expected_output": expected_output[i],
            "model_output": zero_shot_predictions[i]
        },
        output = zero_shot_eval_predictions[i]
    )

    # log few-shot predictions
    comet_llm.log_prompt(
        prompt = system_prompt.format(expected_answers=expected_output[i], predictions=few_shot_predictions[i]),
        tags = ["gpt-3.5-turbo-1106", "few-shot"],
        metadata = {
            "model_name": "gpt-3.5-turbo-1106",
            "temperature": 0,
            "expected_output": expected_output[i],
            "model_output": few_shot_predictions[i]
        },
        output = few_shot_eval_predictions[i]
    )

# Comet View

Results of Few-Shot:

![1-Few-Shot-Tag](images/1-few-shot.png)


Results of Zero-Shot:

![2-Zero-Shot-Tag](images/2-zero-shot.png)